In [ ]:
import json

from StringMatcher import levenshtein_ratio_and_distance
from time import time

#  Load the database into the memory

In [4]:
data = json.load(open("DataStore/datairis.json", "rb"))

#  Load the global secondary index in the memory

In [6]:
GIS = json.load(open("dataStore/datacumulative.json", "rb"))

In [16]:
def getClosestMatch(queryString):
    # Initialize the time counter...
    tic = time()
    
    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(queryString, KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the movieID of the movie which is the best match to the queryString...
    bestMatch_id = list(GIS)[bestMatchRatio_index]
    bestMatch_sepalWidthCm    = GIS[bestMatch_id]

    # Now we have movieID which is the primary key in our main data-store...
    # Retrieve the details of the movie from the MovieID
    irisInfo            = data[str(bestMatch_id)]
    
    # Mark the search completion time...
    toc                  = time()

    # Add max temp to the response...
    irisInfo["SepalWidthCm"] = bestMatch_sepalWidthCm
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : irisInfo
    }
    
    return responseModel

In [14]:
def request(idToSearch=""):
    # Movie name should always be in a string...
    idToSearch = str(idToSearch)
    # Check if the movie name is valid...
    if(idToSearch.replace(" ", "").strip() == ""):
        return f"Invalid iris Name"
    
    # Make a request to fetch the movie info...
    response = getClosestMatch(queryString = idToSearch)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
    # otherwise...
    # Construct user ratings table...
    cumulative = response["response body"]["Id"]
    avocado    = response["response body"]["Id"]

    for kepid in cumulative:
        try    : tags_by_a_single_user_to_a_single_iris = ", ".join([tag for tag in tags if cumulative["Id"] == avocado["Id"]]) # Please don't mind the long veriable name :-)
        except : tags_by_a_single_user_to_a_single_iris = ""
        break
    table.add_row([cumulative["Id"], cumulative["Id"], ", ".join(tags_by_a_single_user_to_a_single_iris), cumulative["Id"]])
    # Now we need to check if a user has given 
    data_to_print = f"\
    Id          : {response['response body']['Id']}\n\
    sepalWidthCm          : {response['response body']['sepalWidthCm']}\n\
    User cumulative   : ---------------------------\n\
    "
    print(data_to_print)
    print(table)

In [ ]:
request(idToSearch="Species")